In [15]:
import duckdb

In [16]:
con=duckdb.connect(database='dados_duckdb.bd', read_only=False)

In [17]:
df=con.execute("""
            SELECT * FROM(SELECT *, ROW_NUMBER() OVER (PARTITION BY NATBR ORDER BY data_ingestao DESC) AS row_num
             FROM bronze_z0019
            where data_ingestao >= '2025-01-11')
            WHERE row_num = 1 
            """).fetchdf()

In [18]:
df.head(100)

,NATBR,MAKTK,WERKS,MAINST,LABST,NOME_ARQUIVO,DATA_INGESTAO,row_num
0,10002,MARTELO,BT50,100,1500,z0019_1.csv,2025-08-11 22:03:01.773730,1
1,10001,PARAFUSO,BT10,100,100,z0019_1.csv,2025-08-11 22:03:01.773730,1
2,10004,SERRA,BT50,100,200,z0019_2.csv,2025-08-11 22:03:01.889456,1
3,10005,MACHADO,BT50,100,100,z0019_2.csv,2025-08-11 22:03:01.889456,1
4,10003,PREGO,BT10,100,50,z0019_2.csv,2025-08-11 22:03:01.889456,1


In [19]:
df.columns

Index(['NATBR', 'MAKTK', 'WERKS', 'MAINST', 'LABST', 'NOME_ARQUIVO',
       'DATA_INGESTAO', 'row_num'],
      dtype='object')

In [20]:
df_final= df.drop(columns=['NOME_ARQUIVO','DATA_INGESTAO','row_num'])
df_final=df_final.rename(columns={'NATBR':'id'})
df_final=df_final.rename(columns={'MAKTK':'nm_produto'})
df_final=df_final.rename(columns={'WERKS':'id_categoria'})
df_final=df_final.rename(columns={'MAINST':'id_fornecedor'})
df_final=df_final.rename(columns={'LABST':'vl_preco'})

In [21]:
df_final.head(100)

,id,nm_produto,id_categoria,id_fornecedor,vl_preco
0,10002,MARTELO,BT50,100,1500
1,10001,PARAFUSO,BT10,100,100
2,10004,SERRA,BT50,100,200
3,10005,MACHADO,BT50,100,100
4,10003,PREGO,BT10,100,50


In [22]:
df2=df_final
df2.dtypes

id               object
nm_produto       object
id_categoria     object
id_fornecedor    object
vl_preco         object
dtype: object

In [23]:
df2=df2.astype({
    'id': 'int32',
    'nm_produto': 'string',
    'id_categoria': 'string',
    'id_fornecedor': 'int32',
    'vl_preco': 'float32'
})

df2.dtypes

id                        int32
nm_produto       string[python]
id_categoria     string[python]
id_fornecedor             int32
vl_preco                float32
dtype: object

In [24]:
con.execute("""
            CREATE TABLE IF NOT EXISTS produtos(
            id BIGINT,
            nm_produto TEXT,
            id_categoria TEXT,
            id_fornecedor BIGINT,
            vl_preco FLOAT
            )
""" )

In [25]:
con.execute("insert into produtos select * from df2")

In [26]:
df_resultado=con.execute("SELECT  * FROM produtos").fetchdf()
df_resultado.head(100)

,id,nm_produto,id_categoria,id_fornecedor,vl_preco
0,10002,MARTELO,BT50,100,1500.0
1,10001,PARAFUSO,BT10,100,100.0
2,10004,SERRA,BT50,100,200.0
3,10005,MACHADO,BT50,100,100.0
4,10003,PREGO,BT10,100,50.0


In [27]:
con.close()